In [5]:
# Manipular os arquivos de vídeo 
import cv2
import os
import threading
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

In [6]:
# Classe responsável pelo conjunto de funções e operações
# do player de vídeo

class VideoPlayer:

    # Método especial responsável por inicializar os atributos do objeto
    # sendo executado automáticamente ao criar um novo objeto da classe.
    # Em Python o método CONSTRUTOR é definido como __init__

    # O self trata-se de uma referência ao próprio objeto.
    # É importante entender que cada objeto criado a partir 
    # de uma classe tem seus próprios atributos, com isso, 
    # o self garante que os métodos da classe possam acessar 
    # e modificar tais atributos.

    def show_controls(self):

        control_window = tk.Tk()
        control_window.title("Controles do Player")
        control_window.geometry("400x600")
        control_window.resizable(False, False)

        # Criando seções de controle
        sections = {
            "Controles de Reprodução": [
                "SPACE - Play/Pause",
                "Q - Sair",
            ],
            "Navegação de Frames": [
                "A - Retroceder 1 frame (Pause)",
                "D - Avançar 1 frame (Pause)"
            ],
            "Velocidade de Reprodução": [
                "1-5 - Ajustar velocidade"
            ],
            "Captura de Frames": [
                "J - Salvar como 'Indolor'",
                "K - Salvar como 'Pouca dor'",
                "L - Salvar como 'Muita dor'"
            ]
        }

        for section, commands in sections.items():
            frame = ttk.LabelFrame(control_window, text=section, padding=10)
            frame.pack(fill="both", expand=True, padx=10, pady=5)

            for command in commands:
                label = tk.Label(frame, text=command, font=("Arial", 10), anchor="w", justify="left")
                label.pack(anchor="w", padx=10, pady=2)

        # Botão para fechar
        close_button = tk.Button(control_window, text="Fechar", command=control_window.destroy)
        close_button.pack(pady=10)

        control_window.mainloop()

    def __init__(self, video_path, fps=30):
        
        self.video_path = video_path # Passa o caminho do objeto
        self.fps = fps # Target FPS
        self.cap = cv2.VideoCapture(self.video_path) # Carrega o arquivo de vídeo

        # Valida se 'cap' recebe corretamente o vídeo

        if not self.cap.isOpened():
            raise ValueError("Erro ao abrir o vídeo.")
        
        self.frame_delay = int(1000 / self.fps) # Tempo de espera entre frames em milissegundos
        self.paused = True # Flag para controlar o pause
        self.frame_index = 0 #Denota a posição inicial do frame
        self.playback_speed = 1  # Velocidade de reprodução padrão
        self.current_frame = None  # Armazena o frame atual

        self.output_dirs = {"J": "Indolor", "K": "Pouca dor", "L": "Muita dor"}
        for directory in self.output_dirs.values():
            os.makedirs(directory, exist_ok=True)

    def update_frame(self):

        self.cap.set(cv2.CAP_PROP_POS_FRAMES, self.frame_index)
        ret, frame = self.cap.read()

        if ret:
            self.current_frame = frame
            cv2.imshow('Video Player', frame)

    def save_frame(self, category):

        if self.current_frame is not None:
            folder = self.output_dirs.get(category)

            if folder:
                filename = os.path.join(folder, f"frame_{self.frame_index}.png")
                cv2.imwrite(filename, self.current_frame)
                print(f"Frame salvo em {filename}")

        else:
            print("Erro: Nenhum frame carregado para salvar.")

    # Função para rodar efetivamente o vídeo
    
    def play(self):
        
        threading.Thread(target=self.show_controls, daemon=True).start()  # Exibir a janela de controles em uma thread separada
        self.update_frame()  # Exibe o primeiro frame antes de entrar no loop
        
        while self.cap.isOpened():
            
            if not self.paused:

                for _ in range(self.playback_speed):  # Avança múltiplos frames conforme a velocidade
                    
                    ret, frame = self.cap.read()
                    
                    if not ret:
                        break
                    
                    self.frame_index = int(self.cap.get(cv2.CAP_PROP_POS_FRAMES))
                    self.current_frame = frame.copy()
                
                cv2.imshow('Video Player', frame)
            
            key = cv2.waitKey(self.frame_delay) & 0xFF

            if key == ord('q'):
                break

            elif key == ord(' '):  # Espaço para pausar/play
                self.paused = not self.paused

            elif self.paused and key == ord('d'):  # Avança um frame
                self.frame_index += 1
                self.update_frame()

            elif self.paused and key == ord('a'):  # Retrocede um frame
                self.frame_index = max(self.frame_index - 1, 0)
                self.update_frame()

            elif key in [ord(str(i)) for i in range(1, 6)]:  # Ajusta a velocidade de reprodução
                self.playback_speed = int(chr(key))

            elif self.paused and key in [ord('j'), ord('k'), ord('l')]:  # Salvar frame pausado
                self.save_frame(chr(key).upper())

        self.cap.release()
        cv2.destroyAllWindows()
    

    def __del__(self):
        if self.cap.isOpened():
            self.cap.release()
            cv2.destroyAllWindows()

In [7]:
# O tkinter é utilizado para a criação de interfaces gráficas
# como janelas, botões, caixas de entrada, etc.
# Neste caso, ele é utilizado para criar a janela que permite
# a escolha do arquivo de vídeo.

root = tk.Tk() # Cria a instância do tkinter
root.withdraw()  # Oculta a janela principal da interface, mantendo apenas o necessário
root.attributes('-topmost', True)  # Força a janela a ficar no topo
root.update()  # Atualiza a janela para aplicar as mudanças

# Trecho responsável por abrir uma janela para a escolha de um arquivo (seu path)

video_path = filedialog.askopenfilename( # A função do tkinter abre o explorador de arquivos
    title="Selecione um vídeo", # Nome da janela a ser aberta
    filetypes=[("Arquivos de vídeo", "*.mp4;*.avi;*.mov;*.mkv")], # Define quais arquivos podem ser escolhidos
    parent=root # Define a janela oculta do tkinter (root) como a janela pai do explorador, com isso
                # as alterações em root afetam a janela do explorador (como manter ela em primeiro plano)
    )

root.destroy() # Elimina a janela após a escolha do arquivo

# Exibe ao usuário do programa se algum arquivo foi ou não escolhido
# de acordo com a presença de um caminho na variável video_path   

if not video_path:
    print("Nenhum arquivo selecionado.")
else:
    print(f"Arquivo selecionado: {video_path}")

Arquivo selecionado: C:/Users/marci_wawp/Desktop/Arquivos/Mestrado/Mestrado Ratos/Videos/WhatsApp Video 2025-01-28 at 13.50.50.mp4


In [8]:

player = VideoPlayer(video_path, fps=30)
player.play()
